In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("spotify_millsongdata.csv")

In [3]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [4]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [5]:
df.shape

(57650, 4)

In [6]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [7]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [8]:
df.head(10)

,artist,song,text
0,Gary Numan,Rip,I'll get down on my knees \r\nI'll do anythin...
1,Our Lady Peace,Hope,Hope climbs atop of the piano \r\nReaching ou...
2,Ofra Haza,Kaddish,"For salvation, Kaddish, \r\nFor redemption, K..."
3,Pat Benatar,Out Of Touch,"I've seen your picture in the paper, on the fr..."
4,Blur,Dying Isn't Easy,This isn't the first time \r\nI've left it to...
5,Lil Wayne,Flowers,"[Lil Wayne] \r\nTha Bizness, tha bizness, tha..."
6,John McDermott,Mother Machree,There's a spot in me heart which no colleen ma...
7,Adele,Cold Shoulder,You say it's all in my head \r\nAnd the thing...
8,Michael W. Smith,Cross My Heart,"Ink and paper, epic offers \r\nGlass moon wal..."
9,Frank Sinatra,Don't Worry 'Bout Me,Don't worry 'bout me \r\nI'll get along \r\n...


In [9]:
df['text'][0]

"I'll get down on my knees  \r\nI'll do anything I can  \r\nI'll even walk upon water  \r\nI'll burn the soul of a man  \r\nI'm so sick with pain  \r\nI'm so sick with need  \r\nI'll lay before the cross and pray  \r\nI've done nothing to you  \r\nI'll bleed like holy rain  \r\nI'll say I'm searching for the truth  \r\nI'll rip the skin from God's face  \r\nIf it can guide me to you  \r\nI'll give life to disease  \r\nI'll steal the light from Heavens gate  \r\nI'll give life to obsession  \r\nAnd then I'll do as I please  \r\nI'll give you one last sin or cure  \r\nI've got something for you  \r\n  \r\n\r\n"

In [10]:
df.shape

(5000, 3)

In [11]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)


In [13]:
pip install nltk

   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 1.5/1.5 MB 13.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [14]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [16]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\peter\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [17]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [19]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [20]:
similarity[0]

array([1.        , 0.05129421, 0.00187822, ..., 0.04421084, 0.03449518,
       0.07659492])

In [21]:
df[df['song'] == 'Crying Over You']

,artist,song,text
345,UB40,Crying Over You,cri over you in the morn cri over you in the e...


In [22]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [23]:
recommendation('Crying Over You')

['Crying',
 'Crying',
 'Cry, Cry, Cry',
 "Don't Cry",
 'A Sentimental Blues',
 'Cry',
 'Cry, Cry, Cry',
 "Don't Cry",
 'Cry Me A River',
 "Don't Cry No Tears",
 "You Won't See Me Cry",
 "Why Can't I Cry",
 'Cry Clock Said',
 'Fool To Cry',
 'Devil Cried',
 'Conehead',
 'Never',
 'Handsome Hands',
 "I Ain't Gonna Cry Tonight",
 "Who's Crying Now"]

In [24]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))